In [5]:
import win32com.client

def list_outlook_stores():
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    stores = outlook.Stores

    print("Stores found in Outlook.Stores:")
    for store in stores:
        print(f"  DisplayName: {store.DisplayName}")
    print()

if __name__ == "__main__":
    list_outlook_stores()


Stores found in Outlook.Stores:
  DisplayName: nihat.yumer@outlook.com
  DisplayName: n.yumer@my.bpp.com
  DisplayName: afizenihat@outlook.com
  DisplayName: nyumer@dlrmn.onmicrosoft.com



In [4]:
import win32com.client

SIGNATURE_TRIGGERS = [
    #"\r\n--",                       # A common email signature delimiter
    "Kind regards", 
    "Best regards", 
    "Sent from my", 
    "Sincerely",
    "Connor",                       # Specific names of the parties involved in hardware collection from the warehouse
    "Stuart",                       # The signature follows the name and there is no need to process the text after the sender's name
    "Nihat",
    "Nelson",
    "Ronnie",
    "Michael",
    "Darren",
    "David",
    "Alan",
    "Ben",
    "Ritesh",
    "Thabani",
]

def strip_signature(body, triggers=SIGNATURE_TRIGGERS):
    """
    My approach: given an email body (string),
    find the earliest occurrence of any 'trigger' phrase
    and return only the text before that occurrence.
    """
    # Normalize to a common case (lower) for searching
    body_lower = body.lower()
    earliest_index = len(body)
    for trigger in triggers:
        trigger_index = body_lower.find(trigger.lower())
        if trigger_index != -1 and trigger_index < earliest_index:
            earliest_index = trigger_index
    return body[:earliest_index].rstrip()

def read_outlook_subfolder_stores(mailbox_display_name, subfolder_name):
    """
    Reads emails from a specific subfolder in the specified mailbox (by Store.DisplayName).

    :param mailbox_display_name: The DisplayName of the Store in Outlook. 
                                 Often the email address, or a friendly name.
    :param subfolder_name: The subfolder under Inbox from which to read emails.
    """

    # Get the MAPI Namespace
    outlook_ns = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

    # Find the specific store by its DisplayName
    target_store = None
    for store in outlook_ns.Stores:
        # Compare case-insensitively
        if store.DisplayName.lower() == mailbox_display_name.lower():
            target_store = store
            break

    if not target_store:
        raise ValueError(f"Mailbox '{mailbox_display_name}' not found in Outlook Stores.")

    # Get the default Inbox folder (olFolderInbox = 6) for this store
    inbox = target_store.GetDefaultFolder(6)

    # Access the specified subfolder under the Inbox
    # If the subfolder is nested deeper, you can chain .Folders[...] calls
    try:
        subfolder = inbox.Folders[subfolder_name]
    except:
        raise ValueError(f"Subfolder '{subfolder_name}' not found under Inbox for '{mailbox_display_name}'.")

    # Get all items (emails) in that subfolder
    messages = subfolder.Items
    # Sort them by ReceivedTime descending (newest first)
    messages.Sort("[ReceivedTime]", True)

    # Read and print the first few emails (e.g. top 5)
    count_to_read = 5
    print(f"--- Reading up to {count_to_read} emails from subfolder '{subfolder_name}' in mailbox '{mailbox_display_name}' ---")
    for i, msg in enumerate(messages, start=1):
        try:
            print(f"Email #{i}")
            print("  From:   ", msg.SenderName)
            print("  Received:", msg.ReceivedTime)           
            raw_body = msg.Body
            body_no_sig = strip_signature(raw_body)
            print("  Body (no signature):\n\n", body_no_sig)
            print("-" * 40)
        except Exception as e:
            print("Error reading message:", e)
        if i >= count_to_read:
           break

if __name__ == "__main__":

    # 1. Specify which mailbox/store to read, exactly matching 'store.DisplayName'
    # mailbox_name = "Nihat.Yumer@nhft.nhs.uk"  
    mailbox_name = "n.yumer@my.bpp.com"  
    subfolder_name = "ROMIC"       # must match exactly a subfolder under the Inbox

    # 2. Read emails from the subfolder
    read_outlook_subfolder_stores(mailbox_name, subfolder_name)


--- Reading up to 5 emails from subfolder 'ROMIC' in mailbox 'n.yumer@my.bpp.com' ---
Email #1
  From:    Ansell Stuart
  Subject: stock out
  Received: 2024-12-20 17:17:20.409000+00:00
  Body (no signature):

 X6 24” screens – MASH @ CJC
----------------------------------------
Email #2
  From:    Melay Connor
  Subject: INC0071514
  Received: 2024-12-20 15:37:20.598000+00:00
  Body (no signature):

 Inventory of Hardware Collected from warehouse  1 x 5540  

  Asset details of devices collected (if required)     Asset 076975
----------------------------------------
Email #3
  From:    Ansell Stuart
  Subject: Stock out
  Received: 2024-12-20 14:30:01.684000+00:00
  Body (no signature):

 X1 3000 - nhft-073807
----------------------------------------
Email #4
  From:    Baramasimbe Ronnie
  Subject: Stock booked out
  Received: 2024-12-20 12:12:07.829000+00:00
  Body (no signature):

 Hi,

 

I have booked out the following items:

 

4 x monitors for SCTASK0035461, SCTASK0031985 and 